In [1]:
%load_ext autoreload
%autoreload 2
import torch
from torch import nn, optim
import numpy as np
import pandas as pd
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import pyro
import pyro.distributions as dist
from pyro.distributions import Categorical, Normal, Multinomial, Binomial, MultivariateNormal, Beta, constraints
from pyro.distributions.torch import Bernoulli
import pyro.infer as infer
from pyro.infer import TraceEnum_ELBO, Trace_ELBO, config_enumerate
from pyro import poutine
from pyro.poutine.trace_messenger import TraceMessenger
from pyro.poutine.messenger import Messenger

plt.style.use('seaborn-v0_8')


/env_nlp/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [15]:
def model():
    # p = pyro.param("p", )
    
    probs_a = torch.softmax(torch.randn(6,), 0)
    a = pyro.sample("a", Categorical(probs=probs_a))
    
    b = pyro.sample("b", Bernoulli(probs_a[a]))

pyro.set_rng_seed(0)
torch.manual_seed(0)
pyro.clear_param_store()
model = config_enumerate(model)
trace = poutine.trace(poutine.enum(model, first_available_dim=-2)).get_trace()
# trace = poutine.trace(poutine.block(model, hide=[])).get_trace()
trace.nodes["a"]

{'type': 'sample',
 'name': 'a',
 'fn': Categorical(probs: torch.Size([6])),
 'is_observed': False,
 'args': (),
 'kwargs': {},
 'value': tensor([[0],
         [1],
         [2],
         [3],
         [4],
         [5]]),
 'infer': {'enumerate': 'parallel',
  'expand': False,
  '_enumerate_dim': -2,
  '_dim_to_id': {-2: 0}},
 'scale': 1.0,
 'mask': None,
 'cond_indep_stack': (),
 'done': True,
 'stop': False,
 'continuation': None}

In [7]:
trace.compute_log_prob()

In [8]:
trace.format_shapes().split("\n")

['Trace Shapes:      ',
 ' Param Sites:      ',
 'Sample Sites:      ',
 '       a dist     |',
 '        value 6 1 |',
 '     log_prob 6 1 |']

In [10]:
trace.nodes["a"]

{'type': 'sample',
 'name': 'a',
 'fn': Categorical(probs: torch.Size([6]), logits: torch.Size([6])),
 'is_observed': False,
 'args': (),
 'kwargs': {},
 'value': tensor([[0],
         [1],
         [2],
         [3],
         [4],
         [5]]),
 'infer': {'enumerate': 'parallel',
  'expand': False,
  '_enumerate_dim': -2,
  '_dim_to_id': {-2: 0}},
 'scale': 1.0,
 'mask': None,
 'cond_indep_stack': (),
 'done': True,
 'stop': False,
 'continuation': None,
 'unscaled_log_prob': tensor([[-0.5232],
         [-2.3576],
         [-4.2429],
         [-1.4957],
         [-3.1487],
         [-3.4627]]),
 'log_prob': tensor([[-0.5232],
         [-2.3576],
         [-4.2429],
         [-1.4957],
         [-3.1487],
         [-3.4627]]),
 'log_prob_sum': tensor(-15.2308)}

In [11]:
trace.nodes["a"]["log_prob"].exp()

tensor([[0.5926],
        [0.0946],
        [0.0144],
        [0.2241],
        [0.0429],
        [0.0313]])

In [13]:
trace.nodes["a"]['fn'].probs

tensor([0.5926, 0.0946, 0.0144, 0.2241, 0.0429, 0.0313])